In [1]:
from collections import defaultdict
import json
from nltk.tokenize import word_tokenize
import numpy as np
import pickle
from factualness import calculate_factualness_score
from topical import calculate_ts_score
from uniqueness import calculate_uniqueness_score
from granularity import calculate_granularity_score
from completeness import calculate_completeness_score
import os

datasets = [
    # 'cdr', 
    # 'docred', 
    'nyt10m', 
    # 'wiki20m', 
    # 'tacred', 
    # 'wiki80'
    ]    

    
if os.path.exists('./evals/nyt10m_gpt4_1106.json'):
    results = json.load(open('./evals/nyt10m_gpt4_1106.json', 'r'))
else:
    results = defaultdict(dict)
    
for dataset_name in datasets:
    print(f'Processing {dataset_name} ...')
    file_name = 'nyt10m_rand_500_gpt-4-1106-preview_1.json'
    file_to_evaluate = f'../processed_results/{file_name}'
    with open(file_to_evaluate, 'r') as f:
        data_to_evaluate = json.load(f)
    
    for text in data_to_evaluate.keys():
        results[text]['extracted_triples'] = data_to_evaluate[text]
        
    print('Computing TS ...')
    dictionary = pickle.load(open(f'./topical_process/{dataset_name}_dictionary.pkl', 'rb'))
    lda_model = pickle.load(open(f'./topical_process/{dataset_name}_lda.pkl', 'rb'))

[nltk_data] Downloading package stopwords to /home/pj20/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pj20/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading element embeddings...
Loading element embeddings...
Loading Ground Truth triple embeddings...
Processing nyt10m ...
Computing TS ...


In [2]:
_, TS_all = calculate_ts_score(data_to_evaluate, dictionary, lda_model, output_all_scores=True)


In [4]:
results = add_scores(results, TS_all, 'TS')

In [5]:
json.dump(results, open('./evals/nyt10m_gpt4_1106.json', 'w'), indent=4)

In [9]:
def add_scores(original, new_scores, attribute_name):
    for text in original.keys():
        if text in new_scores:
            original[text][attribute_name] = new_scores[text]
    return original

In [12]:
print('Computing CS ...')
CS_all = json.load(open(f'./completeness/details/{file_name[:-7]}.json', 'r'))
results = add_scores(results, CS_all, 'CS')
json.dump(results, open('./evals/nyt10m_gpt4_1106.json', 'w'), indent=4)

print(f'Finished processing {dataset_name} ...')

Computing CS ...
Finished processing nyt10m ...
